![Wildcat Eyes](assets/images/wildcat-eyes-logo.jpg)

# Download remote sensing data

## Instructions

* Select a [tile index grid](assets/data/)
* Find the tiles you wish to download and add them the variable, `data_names` as a list
* Select the location on your hard drive where you want to save the data and give the project a name
* Run to download and extract your data!

Note: this version use macOS/Linux command line commands. Select the [Windows OS](windows-version.ipynb) version.


In [ ]:
# Subprocess allows us to issue commands on the command line
import subprocess
# Module to download files with an URL
import urllib.request
# Zip utlity to extract files
from zipfile import ZipFile

In [ ]:
############### Project properties ###############

# Root GIS directory
root = "/Users/boyd"

# Project name - creates a folder in project directory
project = "kyFromAbove"

# 2018, 2016, 2006 (2 ft resolution orthophoto)
# 2014, 2012, 2010, 2008 (1 M resolution orthophoto)
# Some years have 1 ft and 6 inch resolution
# Check tile index grids!
# Specifying a year has no impact on downloading lidar or DEM data
year = "2012" 

data_names = [
"ftp://ftp.kymartian.ky.gov/kyaped/DEMs_5FT/N115E318_DEM.zip",
"ftp://ftp.kymartian.ky.gov/kyaped/DEMs_5FT/N115E319_DEM.zip",
"ftp://ftp.kymartian.ky.gov/kyaped/DEMs_5FT/N115E320_DEM.zip"

]

if f"{year}" not in data_names[0]:
    print("Check your year and tile names. NAIP years 2018, 2016, and 2018 use a 10k grid. The other years use a 20k grid.")

In [ ]:
# Populate project folder with asset folders
downloads = f'{root}/{project}/downloads'
lidar = f'{root}/{project}/lidar'
naip_year = f'{root}/{project}/{year}'
dem = f'{root}/{project}/dem'

# Create folders
folders = [f'{root}/{project}', downloads, lidar, naip_year, dem]
for folder in folders:
    subprocess.run(f'mkdir {folder}', shell=True, stdout=subprocess.PIPE)
    print(f"mkdir {folder}")
    
# Show contents of project directory
completed = subprocess.run(f'ls -la {root}/{project}', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))

In [ ]:
i = 0
names= []

# Downlaoad LAZ
if "laz" in data_names[0][-3:].lower():
    las_names = []
    for url in data_names:
        name = url[-12:]
        las_names.append(f'{lidar}{url[-12:-4]}.las')
        with urllib.request.urlopen(url) as response: 
            print(f'Downloading {url}')
            with open(f'{downloads}{name}', 'wb') as outFile:
                data = response.read()
                outFile.write(data)
        i += 1
# Downlaoad NAIP
elif "naip" in data_names[0][-22:-18].lower():
    extension = "jpg"
    if year == "2016":
        naip_prefix = "ky_2ft_naip_2016_"
        directory = "FSA_NAIP_2016_2FT"
    elif year == "2018":
        naip_prefix = "KY_2FT_NAIP_2018_"
        extension = "tif"
        directory = "FSA_NAIP_2018_2FT"
    elif year == "2006":
        naip_prefix = "ky_2ft_naip_2006_"
        directory = "FSA_NAIP_2006_2FT"
    elif year == "2008":
        naip_prefix = "ky_1m_naip_2008_"
        directory = "FSA_NAIP_2008_1M"
    elif year == "2010":
        naip_prefix = "ky_1m_naip_2010_"
        directory = "FSA_NAIP_2010_1M"
    elif year == "2012":
        naip_prefix = "ky_1m_naip_2012_"
        directory = "FSA_NAIP_2012_1M"
    elif year == "2014":
        naip_prefix = "ky_1m_naip_2014_"
        directory = "FSA_NAIP_2014_1M"
    for url in data_names:
        name = url[-12:-4]
        names.append(name)
        url = f"ftp://ftp.kymartian.ky.gov/{directory}/{naip_prefix}{name}.zip"
        print(f'Downloading {url}')
        with urllib.request.urlopen(url) as response: 
            with open(f'{downloads}/{name}.zip', 'wb') as outFile:
                data = response.read()
                outFile.write(data) 
        with ZipFile(f'{downloads}/{name}.zip', 'r') as zip: 
            zip.extractall(f'{downloads}/{name}')
        print(f'mv {downloads}/{name}/*.* {naip_year}')
        completed = subprocess.run(f'mv {downloads}/{name}/*.* {naip_year}', shell=True, stdout=subprocess.PIPE)
        print(completed.stdout.decode('UTF-8'))
        i += 1
elif "dem" in data_names[0][-7:-4].lower():
    for url in data_names:
        name = url[-16:-4]
        names.append(name)
        url = f"ftp://ftp.kymartian.ky.gov/kyaped/DEMs_5FT/{name}.zip"
        print(f'Downloading {url}')
        with urllib.request.urlopen(url) as response: 
            with open(f'{downloads}/{name}.zip', 'wb') as outFile:
                data = response.read()
                outFile.write(data) 
        with ZipFile(f'{downloads}/{name}.zip', 'r') as zip: 
            zip.extractall(f'{downloads}/{name}')
        print(f'mv {downloads}/{name}/*.* {dem}')
        completed = subprocess.run(f'mv {downloads}/{name}/*.* {dem}', shell=True, stdout=subprocess.PIPE)
        print(completed.stdout.decode('UTF-8'))
        i += 1
elif "1ft" in data_names[0][-16:-13].lower():
    for url in data_names:
        name = url[-12:-4]
        names.append(name)
        url = f"ftp://ftp.kymartian.ky.gov/kyaped/KYAPED_2012_1FT/{name}.zip"
        print(f'Downloading {url}')
        with urllib.request.urlopen(url) as response: 
            with open(f'{downloads}/{name}.zip', 'wb') as outFile:
                data = response.read()
                outFile.write(data) 
        with ZipFile(f'{downloads}/{name}.zip', 'r') as zip: 
            zip.extractall(f'{downloads}/{name}')
        print(f'mv {downloads}/{name}/*.* {year}')
        completed = subprocess.run(f'mv {downloads}/{name}/*.* {naip_year}', shell=True, stdout=subprocess.PIPE)
        print(completed.stdout.decode('UTF-8'))
        i += 1
else:
    for url in data_names:
        name = url[-12:-4]
        names.append(name)
        print(f'Downloading {url}')
        with urllib.request.urlopen(url) as response: 
            with open(f'{downloads}/{name}.zip', 'wb') as outFile:
                data = response.read()
                outFile.write(data) 
        with ZipFile(f'{downloads}/{name}.zip', 'r') as zip: 
            zip.extractall(f'{downloads}/{name}')
        print(f'mv {downloads}/{name}/*.* {year}')
        completed = subprocess.run(f'mv {downloads}/{name}/*.* {naip_year}', shell=True, stdout=subprocess.PIPE)
        print(completed.stdout.decode('UTF-8'))
        i += 1
        
for i in names:
    print(f"Downloaded: {i}")

In [ ]:
# Clean Downloads...
completed = subprocess.run(f'rm -fr {downloads}/*', shell=True, stdout=subprocess.PIPE)
print(completed.stdout.decode('UTF-8'))